In [1]:
import pandas as pds
import numpy as np
import matplotlib.pyplot as plt

Nous allons nous intéresser à une NAP entérobactérienne aux rôles divers et variés :
Fis (Factor for Inversion Stimulation). En tant que protéine architecturale, Fis module
d’une part la topologie de l’ADN de manière directe et indirecte. Fis agit d’autre part
comme un régulateur global de l’expression des gènes en se ﬁxant de manière as-
péciﬁque sur le génome, avec une préférence pour les régions AT riches.

Nous allons partir directement des données de couverture (i.e. nombre de reads
par position génomique) issues d’un séquençage single-end après contrôle qualité et
alignement. Les auteurs ont choisir de retenir les reads pour lesquels l’alignement avec
la séquence de référence était unique, ne présentait pas de gap et moins de 2 mis-
matches. Chaque alignement a été étendu jusqu’à 200 pb du côté 3’, taille moyenne
des fragments d’ADN dans l’expérience.

Nous allons analyser ces données et comparer les résultats obtenus (proﬁl de ﬁxa-
tion de Fis) à ceux publiés dans l’article.

### Fonction de lissage et réduction à moyenne glissante

In [ ]:
def lissage_reduction(x,y,p):
    '''Lissage et réduction par moyenne glissante sur 2p+1 points'''
    xout = np.empty(0) ; yout = np.empty(0)
    for index in range(p, len(y)-p+1, p):
        xout = np.append(xout,np.mean(x[index-p : index+p+1]))
        yout = np.append(yout,np.mean(y[index-p : index+p+1]))
    return xout,yout

 ### Fonction de chargement d'une séquence fasta

In [ ]:
def load_fasta_seq(filename):
    ''' Chargement sequence genomique a partir d'un fichier fasta'''
    seq = str()
    my_file = open(filename, "r") # ouverture fichier fasta
    for i in my_file.readlines(): # parcourt chaque ligne du fichier
        line=i.strip() # supprrime les retours à la ligne
        if line != '': #si la ligne n'est pas vide
            if line[0]!=">": #et si la ligne n'est pas un entete (les entêtes débutent par >)
                seq+=line
    my_file.close()
    return seq

#### Fonction de calcul du %GC

In [ ]:
def GC_content(seq, window=5000):
    ''' Calcul du taux de GC le long du genome '''
    x = [] ; GC = [] # x coordonnees des fenetres
    for i in range(0,len(seq), window//2):
        s = seq[i: i+window]  
        g = s.count('G') + s.count('g') # nb de G
        c = s.count('C') + s.count('c') # nb de C
        GC.append(float(g+c)/len(s))
        x.append(i+1) # coordonnee debut fenetre
    return x,GC